In [1]:
#Load Library
from typing import List
from sentence_transformers import SentenceTransformer
from sentence_transformers import CrossEncoder

import chromadb
from dotenv import load_dotenv

import os
from openai import OpenAI

load_dotenv()

True

In [2]:
#Chunk Function
def split_into_chunks(doc_file: str) -> List[str]:
    with open(doc_file, 'r') as file:
        content = file.read()

    return [chunk for chunk in content.split("\n\n")]

In [3]:
#Embedding Function
def embed_chunk(chunk: str) -> List[float]:
    embedding = embedding_model.encode(chunk, normalize_embeddings=True)
    return embedding.tolist()

In [4]:
#Save to VectorDB Function
def save_embeddings(chunks: List[str], embeddings: List[List[float]]) -> None:
    for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
        chromadb_collection.add(
            documents=[chunk],
            embeddings=[embedding],
            ids=[str(i)]
        )

In [5]:
#Retrieve from VectorDB Function
def retrieve(query: str, top_k: int) -> List[str]:
    query_embedding = embed_chunk(query)
    results = chromadb_collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    return results['documents'][0]

In [6]:
#Rerank Function
def rerank(query: str, retrieved_chunks: List[str], top_k: int) -> List[str]:
    cross_encoder = CrossEncoder('cross-encoder/mmarco-mMiniLMv2-L12-H384-v1')
    pairs = [(query, chunk) for chunk in retrieved_chunks]
    scores = cross_encoder.predict(pairs)

    scored_chunks = list(zip(retrieved_chunks, scores))
    scored_chunks.sort(key=lambda x: x[1], reverse=True)

    return [chunk for chunk, _ in scored_chunks][:top_k]

In [7]:
#Generate Answer Function
def generate(query: str, chunks: List[str]) -> str:
    client = OpenAI(api_key=os.environ.get('DEEPSEEK_API_KEY'), base_url="https://api.deepseek.com")
    
    SYSTEM_PROMPT = f"""你是一位知識助手，请根據用户的問題和下列片段生成淮確的回答。"""
    USER_PROMPT = f"""請使用以下包含在 <context> 標籤內的資訊，來回答包含在 <question> 標籤內的問題。
    
    <context>
    {"\n\n".join(chunks)}
    </context>
    <question>
    {query}
    </question>

    請根據上述提供的内容作答，不要編造信息。
    """
    response = client.chat.completions.create(
        model="deepseek-chat", 
        messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
        ], 
        stream=False)

    return response

In [8]:
#Load Document
chunks = split_into_chunks("1.txt")

#for i, chunk in enumerate(chunks):
#    print(f"[{i}] {chunk}\n")

In [9]:
embedding_model = SentenceTransformer("shibing624/text2vec-base-chinese")

#embedding = embed_chunk("测试内容")
#print(len(embedding))
#print(embedding)

embeddings = [embed_chunk(chunk) for chunk in chunks]

print(len(embeddings))
#print(embeddings[0])

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: shibing624/text2vec-base-chinese
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


1807


In [10]:
chromadb_client = chromadb.EphemeralClient()
chromadb_collection = chromadb_client.get_or_create_collection(name="RAG")

save_embeddings(chunks, embeddings)

In [11]:
query = "關羽的座騎是什麼？"
retrieved_chunks = retrieve(query, 5)

for i, chunk in enumerate(retrieved_chunks):
    print(f"[{i}] {chunk}\n")

[0] 第六十五回 马超大战葭萌关 刘备自领益州牧

[1] 第二十一回 曹操煮酒论英雄 关公赚城斩车胄

[2]     方出城门，只见一人在马前长揖曰：“公所骑马，不可乘也。”玄德视之，乃荆州幕宾伊籍，字机伯，山阳人也。玄德忙下马问之。籍曰：“昨闻蒯异度对刘荆州云：此马名的卢，乘则妨主。因此还公。公岂可复乘之？”玄德曰：“深感先生见爱。但凡人死生有命，岂马所能妨哉！”籍服其高见，自此常与玄德往来。玄德自到新野，军民皆喜，政治一新。建安十二年春，甘夫人生刘禅。是夜有白鹤一只，飞来县衙屋上，高鸣四十余声，望西飞去。临分娩时，异香满室。甘夫人尝夜梦仰吞北斗，因而怀孕，故乳名阿斗。此时曹操正统兵北征。玄德乃往荆州，说刘表曰：“今曹操悉兵北征，许昌空虚，若以荆襄之众，乘间袭之，大事可就也。”表曰：“吾坐据九郡足矣，岂可别图？”玄德默然。表邀入后堂饮酒。酒至半酣，表忽然长叹。玄德曰：“兄长何故长叹？”表曰：“吾有心事，未易明言。”玄德再欲问时，蔡夫人出立屏后。刘表乃垂头不语。须臾席散，玄德自归新野。至是年冬，闻曹操自柳城回，玄德甚叹表之不用其言。忽一日，刘表遣使至，请玄德赴荆州相会。玄德随使而往。刘表接着，叙礼毕，请入后堂饮宴；因谓玄德曰：“近闻曹操提兵回许都，势日强盛，必有吞并荆襄之心。昔日悔不听贤弟之言，失此好机会。”玄德曰：“今天下分裂，干戈日起，机会岂有尽乎？若能应之于后，未足为恨也。”表曰：“吾弟之言甚当。”相与对饮。酒酣，表忽潸然泪下。玄德问其故。表曰：“吾有心事，前者欲诉与贤弟，未得其便。”玄德曰：“兄长有何难决之事？倘有用弟之处，弟虽死不辞。”表曰：“前妻陈氏所生长子琦，为人虽贤，而柔懦不足立事；后妻蔡氏所生少子琼，颇聪明。吾欲废长立幼，恐碍于礼法；欲立长子，争奈蔡氏族中，皆掌军务，后必生乱：因此委决不下。”玄德曰：“自古废长立幼，取乱之道。若忧蔡氏权重，可徐徐削之，不可溺爱而立少子也。”表默然。

[3]     忽一日，操请关公宴。临散，送公出府，见公马瘦，操曰：“公马因何而瘦？”关公曰：“贱躯颇重，马不能载，因此常瘦。”操令左右备一马来。须臾牵至。那马身如火炭，状甚雄伟。操指曰：“公识此马否？”公曰：“莫非吕布所骑赤兔马乎？”操曰：“然也。”遂并鞍辔送与关公。关公再拜称谢。操不悦曰：“吾累送美女金帛，公未尝下拜；今吾赠马，乃喜而再拜：何贱人而贵畜耶？”

In [12]:
reranked_chunks = rerank(query, retrieved_chunks, 3)

for i, chunk in enumerate(reranked_chunks):
    print(f"[{i}] {chunk}\n")

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

XLMRobertaForSequenceClassification LOAD REPORT from: cross-encoder/mmarco-mMiniLMv2-L12-H384-v1
Key                             | Status     |  | 
--------------------------------+------------+--+-
roberta.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


[0]     忽一日，操请关公宴。临散，送公出府，见公马瘦，操曰：“公马因何而瘦？”关公曰：“贱躯颇重，马不能载，因此常瘦。”操令左右备一马来。须臾牵至。那马身如火炭，状甚雄伟。操指曰：“公识此马否？”公曰：“莫非吕布所骑赤兔马乎？”操曰：“然也。”遂并鞍辔送与关公。关公再拜称谢。操不悦曰：“吾累送美女金帛，公未尝下拜；今吾赠马，乃喜而再拜：何贱人而贵畜耶？”关公曰：“吾知此马日行千里，今幸得之，若知兄长下落，可一日而见面矣。”操愕然而悔。关公辞去。后人有诗叹曰：“威倾三国著英豪，一宅分居义气高。奸相枉将虚礼待，岂知关羽不降曹。”操问张辽曰：“吾待云长不薄，而彼常怀去心，何也？”辽曰：“容某探其情。”次日，往见关公。礼毕，辽曰：“我荐兄在丞相处，不曾落后？”公曰：“深感丞相厚意。只是吾身虽在此，心念皇叔，未尝去怀。”辽曰：“兄言差矣，处世不分轻重，非丈夫也。玄德待兄，未必过于丞相，兄何故只怀去志？”公曰：“吾固知曹公待吾甚厚。奈吾受刘皇叔厚恩，誓以共死，不可背之。吾终不留此。要必立效以报曹公，然后去耳。”辽曰：“倘玄德已弃世，公何所归乎？”公曰：“愿从于地下。”辽知公终不可留，乃告退，回见曹操，具以实告。操叹曰：“事主不忘其本，乃天下之义士也！”荀彧曰：“彼言立功方去，若不教彼立功，未必便去。”操然之。却说玄德在袁绍处，旦夕烦恼。绍曰：“玄德何故常忧？”玄德曰：“二弟不知音耗，妻小陷于曹贼；上不能报国，下不能保家：安得不忧？”绍曰：“吾欲进兵赴许都久矣。方今春暖，正好兴兵。”便商议破曹之策。田丰谏曰：“前操攻徐州，许都空虚，不及此时进兵；今徐州已破，操兵方锐，未可轻敌。不如以久持之，待其有隙而后可动也。”绍曰：“待我思之。”因问玄德曰：“田丰劝我固守，何如！”玄德曰：“曹操欺君之贼，明公若不讨之，恐失大义于天下。”绍曰：“玄德之言甚善。”遂欲兴兵。田丰又谏。绍怒曰：“汝等弄文轻武，使我失大义！”田丰顿首曰：“若不听臣良言，出师不利。”绍大怒，欲斩之。玄德力劝，乃囚于狱中，沮授见田丰下狱，乃会其宗族，尽散家财，与之诀曰：“吾随军而去，胜则威无不加，败则一身不保矣！”众皆下泪送之。

[1]     方出城门，只见一人在马前长揖曰：“公所骑马，不可乘也。”玄德视之，乃荆州幕宾伊籍，字机伯，山阳人也。玄德忙下马问之。籍曰：“昨闻蒯异度对刘荆州云：此马名的卢，乘则妨主。因

In [13]:
answer = generate(query, reranked_chunks)

In [14]:
print(answer.choices[0].message.content)

根據提供的內容，關羽的座騎是曹操贈送的赤兔馬。
